In [5]:
import time
import random
import copy
import pandas as pd

In [2]:
def read_instance(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    n = int(lines[0].strip())
    capacity = int(lines[1].strip())
    objects = []

    for line in lines[2:]:
        values = list(map(int, line.strip().split()))
        objects.append(values)

    return n, capacity, objects

In [3]:
def evaluate_solution(solution, objects):
    total_value = sum(solution[i] * objects[i][0] for i in range(len(solution)))
    return total_value

In [4]:
def heuristic_constructive(n, capacity, objects):
    start_time = time.time()

    sorted_objects = sorted(objects, key=lambda x: x[0] / x[1], reverse=True)

    current_capacity = 0
    solution = [0] * n

    for i in range(n):
        if current_capacity + sorted_objects[i][0] <= capacity:
            solution[i] = 1
            current_capacity += sorted_objects[i][0]

    selected_items = [(sorted_objects[i][0], sorted_objects[i][1]) for i in range(n) if solution[i] == 1]
    print("Selected Items and Their Weights:", selected_items)

    execution_time = time.time() - start_time

    return solution, execution_time

In [10]:
def tabu_search(n, capacity, objects, max_iterations, tabu_size):
    current_solution, _ = heuristic_constructive(n, capacity, objects)
    current_value = evaluate_solution(current_solution, objects)

    start_time = time.time()

    tabu_list = []

    for _ in range(max_iterations):
        best_neighbor_solution = None
        best_neighbor_value = float('-inf')

        for i in range(len(current_solution)):
            # Try removing the item
            neighbor_solution = copy.deepcopy(current_solution)
            neighbor_solution[i] = 1 - neighbor_solution[i]  # Flip the value
            neighbor_value = evaluate_solution(neighbor_solution, objects)

            if (
                sum(objects[j][1] for j in neighbor_solution) <= capacity
                and neighbor_value > best_neighbor_value
                and neighbor_solution not in tabu_list
            ):
                best_neighbor_solution = neighbor_solution
                best_neighbor_value = neighbor_value

        if best_neighbor_solution is not None:
            current_solution = best_neighbor_solution
            current_value = best_neighbor_value
            tabu_list.append(best_neighbor_solution)
            if len(tabu_list) > tabu_size:
                tabu_list.pop(0)  # Remove the oldest solution from the tabu list

    execution_time = time.time() - start_time

    return current_solution, execution_time

In [12]:
file_path = "testPSAD/test100-SC(1).txt"
n, capacity, objects = read_instance(file_path)
max_iterations = 100
tabu_size = 5
tabu_solution, tabu_time = tabu_search(n, capacity, objects, max_iterations, tabu_size)


Selected Items and Their Weights: [(11, 1), (13, 3), (16, 6), (17, 7), (17, 7), (17, 7)]
